In [98]:
%%sql
SELECT t.USER_ID 
     , t.TRANSACTION_DATE 
     , t.TRANSACTION_TYPE 
     , t.VALUE 
     , tt.DESCRIPTION 
  FROM ORACLE1.OT.TRANSACTIONS t
  JOIN ORACLE1.OT.TRANSACTION_TYPES tt ON t.CATEGORY_ID = tt.ID 
 ORDER BY t.USER_ID
 LIMIT 10

,USER_ID,TRANSACTION_DATE,TRANSACTION_TYPE,VALUE,DESCRIPTION
0,1.000000000000000000,2020-03-04 19:20:22,DEBITO,253.0800000000,Outros
1,1.000000000000000000,2020-03-29 21:51:10,CREDITO,413.1600000000,Educação
2,1.000000000000000000,2020-03-10 05:51:52,DEBITO,570.5300000000,Outros
3,1.000000000000000000,2020-03-30 02:05:05,DEBITO,721.3500000000,Serviços
4,1.000000000000000000,2020-03-25 06:48:34,CREDITO,287.6800000000,Outros
5,22.000000000000000000,2020-03-20 10:52:58,CREDITO,62.7600000000,Outros
6,23.000000000000000000,2020-03-01 13:34:02,CREDITO,280.2600000000,Serviços
7,23.000000000000000000,2020-03-23 01:28:24,CREDITO,773.8600000000,Transporte
8,23.000000000000000000,2020-03-14 12:38:09,CREDITO,258.4300000000,Educação
9,23.000000000000000000,2020-03-12 16:22:23,CREDITO,38.0300000000,Transporte


In [124]:
%%sql
CREATE TABLE CHURN AS
SELECT a.postcode `CEP`
     , e._c1 `ESTADO`
     , u.gender `GENERO`
     , substring(u.cell,2,2) `DDD`
     , substring(replace(u.cell,'-',''),6) `CELULAR`
     , datediff(now(), u.dob)/365 `IDADE`
     , datediff(now(), l.registered)/365 `IDADE_CONTA`
     , cast(nvl(c.total, 0) as float) `CREDITO`
     , cast(nvl(d.total, 0) as float) `DEBITO`
     , cast(nvl(ts.total, 0) as float) `T_SERVICOS`
     , cast(nvl(td.total, 0) as float) `T_EDUCACAO`
     , cast(nvl(tr.total, 0) as float) `T_RESTAURANTE`
     , cast(nvl(tt.total, 0) as float) `T_TRANSPORTE`
     , cast(nvl(tl.total, 0) as float) `T_LAZER`
     , cast(nvl(tm.total, 0) as float) `T_SUPERMERCADO`
     , cast(nvl(to.total, 0) as float) `T_OUTROS`
     , if(l.inactivate_date is null, 'False.', 'True.') `CHURN`
  FROM fin_br.fin.LOGIN l 
  JOIN fin_br.fin.USERS u ON l.UUID = u.UUID 
  JOIN fin_br.fin.ADDRESS a ON a.UUID = u.UUID 
  JOIN csv.`workspace://churn/data/estados.csv.gz` e ON upper(e._c0) = upper(a.state)
  LEFT OUTER JOIN CREDITO c ON c.USER_ID = l.UUID
  LEFT OUTER JOIN DEBITO d ON d.USER_ID = l.UUID
  LEFT OUTER JOIN T_SERVICOS ts ON ts.USER_ID = l.UUID
  LEFT OUTER JOIN T_EDUCACAO td ON td.USER_ID = l.UUID
  LEFT OUTER JOIN T_RESTAURANTE tr ON tr.USER_ID = l.UUID
  LEFT OUTER JOIN T_TRANSPORTE tt ON tt.USER_ID = l.UUID
  LEFT OUTER JOIN T_LAZER tl ON tl.USER_ID = l.UUID
  LEFT OUTER JOIN T_SUPERMERCADO tm ON tm.USER_ID = l.UUID
  LEFT OUTER JOIN T_OUTROS to ON to.USER_ID = l.UUID
 WHERE c.total IS NOT NULL
    OR d.total IS NOT NULL

""


In [91]:
%%sql
CREATE OR REPLACE VIEW CREDITO AS
SELECT t.USER_ID
     , SUM(t.VALUE) `TOTAL`
  FROM ORACLE1.OT.TRANSACTIONS t
 WHERE t.TRANSACTION_TYPE = 'CREDITO'
 GROUP BY t.USER_ID

""


In [89]:
%%sql
CREATE OR REPLACE VIEW DEBITO AS
SELECT t.USER_ID
     , SUM(t.VALUE) `TOTAL`
  FROM ORACLE1.OT.TRANSACTIONS t
 WHERE t.TRANSACTION_TYPE = 'DEBITO'
 GROUP BY t.USER_ID

""


In [102]:
%%sql
select * from ORACLE1.OT.TRANSACTION_TYPES order by 1

,ID,DESCRIPTION
0,1.000000000000000000,Serviços
1,2.000000000000000000,Educação
2,3.000000000000000000,Transporte
3,4.000000000000000000,Restaurante
4,5.000000000000000000,Lazer
5,6.000000000000000000,Supermercado
6,7.000000000000000000,Outros


In [103]:
%%sql
CREATE OR REPLACE VIEW T_SERVICOS AS
SELECT t.USER_ID
     , SUM(t.VALUE) `TOTAL`
  FROM ORACLE1.OT.TRANSACTIONS t
 WHERE t.CATEGORY_ID = 1
 GROUP BY t.USER_ID

""


In [101]:
%%sql
CREATE OR REPLACE VIEW T_EDUCACAO AS
SELECT t.USER_ID
     , SUM(t.VALUE) `TOTAL`
  FROM ORACLE1.OT.TRANSACTIONS t
 WHERE t.CATEGORY_ID = 2
 GROUP BY t.USER_ID

""


In [105]:
%%sql
CREATE OR REPLACE VIEW T_RESTAURANTE AS
SELECT t.USER_ID
     , SUM(t.VALUE) `TOTAL`
  FROM ORACLE1.OT.TRANSACTIONS t
 WHERE t.CATEGORY_ID = 3
 GROUP BY t.USER_ID

""


In [106]:
%%sql
CREATE OR REPLACE VIEW T_TRANSPORTE AS
SELECT t.USER_ID
     , SUM(t.VALUE) `TOTAL`
  FROM ORACLE1.OT.TRANSACTIONS t
 WHERE t.CATEGORY_ID = 4
 GROUP BY t.USER_ID

""


In [107]:
%%sql
CREATE OR REPLACE VIEW T_LAZER AS
SELECT t.USER_ID
     , SUM(t.VALUE) `TOTAL`
  FROM ORACLE1.OT.TRANSACTIONS t
 WHERE t.CATEGORY_ID = 5
 GROUP BY t.USER_ID

""


In [108]:
%%sql
CREATE OR REPLACE VIEW T_SUPERMERCADO AS
SELECT t.USER_ID
     , SUM(t.VALUE) `TOTAL`
  FROM ORACLE1.OT.TRANSACTIONS t
 WHERE t.CATEGORY_ID = 6
 GROUP BY t.USER_ID

""


In [109]:
%%sql
CREATE OR REPLACE VIEW T_OUTROS AS
SELECT t.USER_ID
     , SUM(t.VALUE) `TOTAL`
  FROM ORACLE1.OT.TRANSACTIONS t
 WHERE t.CATEGORY_ID = 7
 GROUP BY t.USER_ID

""
